In [48]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles, quickVizNumpy
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin, sqrt, arctan2, array
import torch.nn as nn
from einops import rearrange, repeat
### fucking around
# from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
from train import getSpacing, getTargetCumulativeTransmittance, getTargetTerminationDistribution


In [75]:
path = r'/home/ansonhon/anson/thesis/LiDAR_NeRF/datasets/training_rainbow/round_plant2.npy'
x = np.load(path)


In [78]:
quickVizNumpy(x[0:1000000,:])

In [65]:
path = r'/home/ansonhon/anson/thesis/LiDAR_NeRF/datasets/training_euclidean_uncentered/round_plant2.npy'
data = np.load(path)
data.shape

(6932455, 9)

In [66]:
pos = data[:,6:9]
org = data[:,0:3]
dir = data[:,3:6]

centroid = np.mean(pos, axis=0)
centroid_tiled = repeat(centroid, 'b -> n b', n = pos.shape[0])

In [67]:
new_pos = pos-centroid_tiled

In [68]:
max_diff = np.max(new_pos, axis=0) - np.min(new_pos, axis=0)

In [85]:
test = rearrange(max_diff, 'a -> a 1')

In [92]:
a = test.T


(1, 3)
(3, 1)


In [71]:
new_pos /= (0.9024175*0.5)


In [81]:
np.min(new_pos, axis=0)

array([-1.03084655, -0.91945115, -0.45719922])

In [74]:
np.save('round_plant2.npy',new_pos)

In [100]:
import numpy as np

def filter_rows_with_min_third_column(data):
    # Create a structured array to facilitate finding unique rows
    dtype = [('first', data.dtype), ('second', data.dtype), ('third', data.dtype)]
    structured_data = np.array(list(map(tuple, data)), dtype=dtype)

    # Sort by first, second, and third columns to ensure the smallest third value comes first
    structured_data.sort(order=['first', 'second', 'third'])
    print(structured_data)

    # Use np.unique to find unique pairs in the first and second columns
    x, unique_indices = np.unique(structured_data[['first', 'second']], return_index=True)
    print

    # Return the filtered array, using the unique indices
    return data[unique_indices]

# Example usage
example_data = np.array([
    [1, 2, 5],
    [1, 2, 3],
    [1, 2, 4],
    [3, 4, 7],
    [3, 4, 5],
    [5, 6, 1],
    [5, 6, 2]
])

filtered_data = filter_rows_with_min_third_column(example_data)
print(filtered_data)


[(1, 2, 3) (1, 2, 4) (1, 2, 5) (3, 4, 5) (3, 4, 7) (5, 6, 1) (5, 6, 2)]
[(1, 2) (3, 4) (5, 6)]
[[1 2 5]
 [3 4 7]
 [5 6 1]]


In [111]:
example_data = np.array([
    [1, 2, 5],
    [1, 2, 3],
    [1, 2, 4],
    [3, 4, 7],
    [3, 4, 5],
    [5, 6, 1],
    [5, 6, 2]
])

example_data = example_data[example_data[:,2].argsort()]
example_data = example_data[example_data[:,1].argsort()]
example_data = example_data[example_data[:,0].argsort()]
_ , unique_indicies = np.unique(example_data[:,0:2], axis=0, return_index = True)
print(example_data)
example_data = example_data[unique_indicies]
print(example_data)



[[1 2 3]
 [1 2 4]
 [1 2 5]
 [3 4 5]
 [3 4 7]
 [5 6 1]
 [5 6 2]]
[[1 2 3]
 [3 4 5]
 [5 6 1]]


In [109]:
unique_indicies

array([0, 3, 5])